# 16. Which player has the highest winning percentage against top 10 ranked opponents?

In [2]:
import pandas as pd

MatchHomeTeamInfo = pd.read_csv(r"E:\Data Analysis Course\Data analyse\tennis_data_analysis\data\raw\MatchHomeTeamInfo.csv")
MatchAwayTeamInfo = pd.read_csv(r"E:\Data Analysis Course\Data analyse\tennis_data_analysis\data\raw\MatchAwayTeamInfo.csv")
MatchEventInfo = pd.read_csv(r"E:\Data Analysis Course\Data analyse\tennis_data_analysis\data\raw\MatchEventInfo.csv")

# Merge MatchEventInfo with MatchHomeTeamInfo and MatchAwayTeamInfo to get player IDs
merged = MatchEventInfo[['match_id', 'winner_code']].merge(MatchHomeTeamInfo[['match_id', 'player_id' , 'current_rank']], on='match_id', suffixes=('', '_home'))
merged = merged.merge(MatchAwayTeamInfo[['match_id', 'player_id' , 'current_rank']], on='match_id', suffixes=('', '_away')).drop_duplicates(subset="match_id")

# Map winner_code to actual player_id
merged['winner_id'] = merged.apply(
    lambda row: row['player_id'] if row['winner_code'] == 1.0 else row['player_id_away'], 
    axis=1
)

merged['loser_id'] = merged.apply(
    lambda row: row['player_id_away'] if row['winner_code'] == 1.0 else row['player_id'], 
    axis=1
)
merged['loser_rank'] = merged.apply(
    lambda row: row['current_rank_away'] if row['winner_code'] == 1.0 else row['current_rank'], 
    axis=1
)

total_wins = merged.groupby('winner_id' , as_index=False)["match_id"].count()
total_wins = total_wins.rename(columns={'match_id': 'total_wins'})

merged_top_10 = merged[(merged['loser_rank'] <= 10)]

top_wins = merged_top_10.groupby('winner_id' , as_index=False)["match_id"].count()
top_wins = top_wins.rename(columns={'match_id': 'top_wins'})

merge_total_top = pd.merge(total_wins, top_wins, on='winner_id', how='inner')

merge_total_top['top_win_percentage'] = (merge_total_top['top_wins'] / merge_total_top['total_wins']) * 100

# Find the winner_id with the max percentage of top wins
max_percentage_winner_id = merge_total_top.loc[merge_total_top['top_win_percentage'].idxmax(), 'winner_id']
max_percentage = merge_total_top['top_win_percentage'].max()

print(f"Winner ID with the maximum percentage of top wins: {max_percentage_winner_id}")
print(f"Maximum percentage of top wins: {max_percentage:.2f}%")


Winner ID with the maximum percentage of top wins: 14486.0
Maximum percentage of top wins: 50.00%
